# M2Xへの接続

## 参考
* [M2X API Documentation(AT&T)](https://m2x.att.com/developer/documentation/v2/overview)
  * M2Xの提供するAPIに関する公式ドキュメント
* [How to connect to the AT&T M2X service from a Raspberry Pi](https://m2x.att.com/developer/tutorials/raspberry/)
  * ラズベリー・パイからM2Xに接続するためのソフトウェアセットアップ手順
* [m2x-python(AT&T's M2X Python Client)](https://github.com/attm2x/m2x-python)
  * PythonでM2Xに接続するためのライブラリ
  * 最初は[README](https://github.com/attm2x/m2x-python/blob/master/README.md)および[USAGE](https://github.com/attm2x/m2x-python/blob/master/USAGE.md)を参照
  * 使い方については[examples](https://github.com/attm2x/m2x-python/tree/master/examples)にあるサンプルプログラムが参考になる
* [AT&T M2X Python Client Library](https://m2x-python.readthedocs.io/en/latest/)
  * m2x-pythonの紹介ページ

In [ ]:
# モジュールのインポート
from m2x.client import M2XClient
from m2x.v2.devices import Device

アカウント設定ページから Master Key を調べる

https://m2x.att.com/account

In [ ]:
# アクセス用のオブジェクトを生成する
client = M2XClient(key='<マスターキー>')

In [ ]:
# 新しいデバイスを生成する
# https://github.com/attm2x/m2x-python/blob/master/examples/create_device.py
device = client.create_device(
    name='Current Time Example',
    description='Store current time',
    visibility='public'
)

print(device.id)

In [ ]:
# デバイスを探す
# https://github.com/attm2x/m2x-python/blob/master/examples/search_devices.py

params = {
    "name": 'Current Time Example'
}

# デバイスの検索．
response = Device.search(api=client, data=params)

for device in response:
    print("Name:", device.name)
    print("ID:", device.id)

In [ ]:
# デバイスを調べる
for dev in client.devices():
    print("device.name: ", dev.name)
    print("  id:", dev.id)
    print("  key:", dev.key)
    print("  visibility:", dev.visibility)
    print("  description:", dev.description)

In [ ]:
# デバイスの新しいストリームを生成する
# https://github.com/attm2x/m2x-python/blob/master/examples/create_stream.py

params = {
    "name": 'Current Time Example'
}
device = Device.search(api=client, data=params)[0]
stream = device.create_stream('curTime')
print("stream name:", stream.name)
print("stream id:", stream.name)

In [ ]:
# データをポストする
# https://github.com/attm2x/m2x-python/blob/master/examples/post_value.py
import time

try:
    while True:
        t = time.time()
        stream.add_value(int(t))
        print("Posted data: ", t)
        time.sleep(4)
except KeyboardInterrupt:
    print("Keyboard Interrupt. Abort.")


In [56]:
# データをエクスポートする
# https://github.com/attm2x/m2x-python/blob/master/examples/export_values.py
params = {
    "name": 'Current Time Example'
}
device = Device.search(api=client, data=params)[0]
stream = device.create_stream('curTime')

res = device.values_export()
print("res:", res)

location = res.headers["location"]

print("The export job url is %s" % location)

job = client.job(location.split("/")[-1])

state = job.state

while(state != "complete"):

    if(state == "failed"): sys.exit("Job has failed to complete")

    print("Job state is %s, waiting for completion" % state)

    time.sleep(5)

    job = client.job(location.split("/")[-1])

    state = job["state"]

print("The job has been completed!\nYou can download the result from %s" % job.result["url"])

res: <m2x.api.HTTPResponse object at 0x10cf7a518>
The job location is in https://api-m2x.att.com/v2/jobs/2018111b1e829d657dbee61b82a0e95335e664
Job state is queued, waiting for completion
The job has been completed! You can download the result from http://export.m2x.sl.attcompute.com/d4efeb2db43df9e8c013376d29a9abbf%20-%20Current%20Time%20Example%20-%202018-11-12T20:52:24.695Z.csv


In [ ]:
# ダウンロードのURLをもとに，データをダウンロードする
import requests

print("download from ", job.result["url"])
response = requests.get(job.result["url"])

In [ ]:
# ダウンロードしたテキストデータの中身を確認
print(response.text)

In [ ]:
# データをCSVファイルに保存
with open('some.csv', 'w') as f:
    f.write(response.text)

In [ ]:
# 行毎に分割してリストにする
response.text.split()

In [ ]:
# 先頭行を除く
response.text.split()[1:]

In [ ]:
# 各行をカンマで分割して2要素のリストに
csv_string=[line.rstrip().split(",") for line in response.text.split()[1:]]

print(csv_string)

In [ ]:
# 以上より response.text からグラフにプロットするまでのまとめ

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt


x = []
y = []
for item in [line.rstrip().split(",") for line in response.text.split()[1:]]:
    x.append(item[0])
    y.append(item[1])

plt.plot(x, y, marker="o")
plt.show()
